# Introduction:

This notebook will experiment with a character-based word prediction task. We will be following the tutorial here: https://machinelearningmastery.com/develop-character-based-neural-language-model-keras/


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline  

import numpy as np

# alphabet
import string

In [2]:
# load the data
import os,sys,re
from google.colab import files
from google.colab import drive

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving six_pence.txt to six_pence (1).txt
User uploaded file "six_pence.txt" with length 412 bytes


In [105]:
text_chunk = 'six_pence.txt'
poem = uploaded[text_chunk].decode("utf-8")
# strip away punctuation; keep spaces!
poem_string = [i.replace('.','') for i in poem.split('\n') if i is not '']
poem_string = [i.replace(',','') for i in poem_string]
poem_string = [i.replace(';','') for i in poem_string]
poem_string = [i.replace('\'','') for i in poem_string]

# poem_string = [i for i in poem.split('\n') if i is not '']
# poem_string = [i.replace(' ','') for i in poem_string]
poem_string = ' '.join(poem_string)
# lower case everything
poem_string = ''.join([i.lower() for i in poem_string])
print(poem_string)

sing a song of sixpence a pocket full of rye four and twenty blackbirds baked in a pie when the pie was opened the birds began to sing wasnt that a dainty dish to set before the king the king was in his counting house counting out his money the queen was in the parlour eating bread and honey the maid was in the garden hanging out the clothes when down came a blackbird and pecked off her nose


In [0]:
# one-hot encoding of characters
def one_hot_character(instr,addspace=True,add_punctuation=True):
  # generate the alphabet
  english_alpha = string.ascii_lowercase
  # if addspace, then incorporate into the alphabet
  if addspace:
    english_alpha = english_alpha+' ' # add the space
  if add_punctuation:
    english_alpha = english_alpha+',.;\'' # add punctuation
  # now, generate one-hot encoding vectors...
#   source_Vector = np.zeros(len(english_alpha))
  one_hot = []
  for i,j in enumerate(instr):
    # alphabet index
    try:
      alpha_ind = english_alpha.index(j)
      source_Vector = np.zeros(len(english_alpha))
      np.put(source_Vector,alpha_ind,float(1))
      one_hot.append(source_Vector)
    except IOError:
      continue
    else:
      continue
  return(one_hot)

def alphabet_index_encode(instr,addspace=True,add_punctuation=True):
  # generate the alphabet
  english_alpha = string.ascii_lowercase
  if addspace:
    english_alpha = english_alpha+' ' # add the space
  if add_punctuation:
    english_alpha = english_alpha+',.;\'' # add punctuation
  # now, generate one-hot encoding vectors...
#   source_Vector = np.zeros(len(english_alpha))
  ind_encode = []
  for i,j in enumerate(instr):
    # alphabet index
    try:
      alpha_ind = english_alpha.index(j)
      ind_encode.append(alpha_ind)
    except IOError:
      continue
    else:
      continue
  return(ind_encode)


def make_string_tensor(instr,addspace,add_punctuation,N_OUTPUT=1,N_INPUT=10):
#   N_OUTPUT = 1
#   N_INPUT = 10
  total_len = N_INPUT + N_OUTPUT

  # one-hot encode the poem
#   poem_string_onehot = one_hot_character(poem_string)
  poem_string_onehot = alphabet_index_encode(poem_string,addspace=addspace,add_punctuation=add_punctuation)
  
  # tile the string:
  string_tiles = []
  for i in range(total_len,len(poem_string_onehot)+1):
    j = poem_string_onehot[i - total_len:i]
    # turn this into a tensor:
    input_tensor = torch.FloatTensor(j[:-1])
    print(j)
    print(input_tensor)
    output_tensor = torch.FloatTensor([j[-1]])#,dtype=torch.float64)
    print(output_tensor)
    string_tiles.append([input_tensor,output_tensor])
  return(string_tiles)

def make_onehot_string_tensor(instr,addspace,add_punctuation,N_OUTPUT=1,N_INPUT=10):
#   N_OUTPUT = 1
#   N_INPUT = 10
  total_len = N_INPUT + N_OUTPUT

  # one-hot encode the poem
#   poem_string_onehot = one_hot_character(poem_string)
  poem_string_onehot = one_hot_character(poem_string,addspace=addspace,add_punctuation=add_punctuation)
  
  # tile the string:
  string_tiles = []
  for i in range(total_len,len(poem_string_onehot)+1):
    j = poem_string_onehot[i - total_len:i]
    # turn this into a tensor:
    input_tensor = torch.FloatTensor(j[:-1])
#     print(j)
#     print(input_tensor)
    output_tensor = torch.FloatTensor([j[-1]])#,dtype=torch.float64)
#     print(output_tensor)
    string_tiles.append([input_tensor,output_tensor])
  return(string_tiles)



def decode_string_vec(invec,addspace=True):
  # generate the alphabet
  english_alpha = string.ascii_lowercase
  # if addspace, then incorporate into the alphabet
  if addspace:
    english_alpha = english_alpha+' ' # add the space
  #
  outchar = english_alpha[list(invec).index(1)]
  return(outchar)
  

In [0]:
## INPUT preparation
# one-hot encode the poem string
# break the text chunks into N_INPUT input characters and N_OUTPUT output characters
# poem_string_input = make_string_tensor(poem_string,addspace=True,add_punctuation=False)
# poem_string_input = make_string_tensor(poem_string)

# cast as float64
# tensor.cast(poem_string_input, tf.float64)
# # test decoding!
# test_decode_out = []
# for i in poem_string_input:
#   outvec = i[1]
#   test_decode_out.append(decode_string_vec(outvec))

In [125]:
print(poem_string_input[1])
print(len(poem_string_input))

[tensor([ 8., 13.,  6., 26.,  0., 26., 18., 14., 13.,  6.]), tensor([26.])]
384


In [0]:
# generate trainloader and testloader sets
BATCH_SIZE = 12

# # list all transformations
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.1307,), (0.3081,))])

# # download and load training dataset
# trainset = torchvision.datasets.MNIST(root='./data', train=True,
#                                         download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(poem_string_input, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

# # download and load testing dataset
# testset = torchvision.datasets.MNIST(root='./data', train=False,
#                                        download=True, transform=transform)
testloader = torch.utils.data.DataLoader(poem_string_input, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)


In [129]:
print(trainloader)

In [0]:
# character/word RNN
# include a method in the class to perform one-hot encoding of inputs and outputs.

class PRNN(nn.Module):
    def __init__(self, n_inputs, n_hidden,n_output,mr):
        super(PRNN, self).__init__()
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        self.mr = mr
        self.encoder = nn.Linear(n_inputs,n_hidden)
        self.recurrent = nn.Linear(n_hidden,n_hidden)
        self.decoder = nn.Linear(n_hidden, n_output)
        self.rnn = nn.RNN(n_inputs, n_hidden)
        
    def forward(self, x0):
        x0=x0.permute(1,0,2)
        self.h1 = torch.zeros(1,BATCH_SIZE,self.n_hidden)
        #self.h1 = Variable(torch.zeros(x0.size(0), self.n_hidden))
        #for t in range(T):
           #self.h1 = self.mr*self.h1+(1-self.mr)*torch.relu(self.encoder(x0[:,t+7,:])+self.recurrent(self.h1))
        self.y0, self.h1 = self.rnn(x0,self.h1)
        self.y1 = self.decoder(self.h1[0])
        
        return self.y1
# can I modify this to set an arbitrary number of layers?
        
# RNN LSTM  
class PRNN_LSTM(nn.Module):
    def __init__(self, n_inputs, n_hidden,n_output,mr):
        super(PRNN_LSTM, self).__init__()
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        self.mr = mr
#         self.encoder = nn.Linear(n_inputs,n_hidden)
#         self.recurrent = nn.Linear(n_hidden,n_hidden)
        self.decoder = nn.Linear(n_hidden, n_output)
        self.lstm = nn.LSTM(n_inputs, n_hidden)
        
    def forward(self, x0):
        x0=x0.permute(1,0,2)
        self.h1 = (torch.zeros(1,BATCH_SIZE,self.n_hidden),torch.zeros(1,BATCH_SIZE,self.n_hidden))
        #self.h1 = Variable(torch.zeros(x0.size(0), self.n_hidden))
        #for t in range(T):
           #self.h1 = self.mr*self.h1+(1-self.mr)*torch.relu(self.encoder(x0[:,t+7,:])+self.recurrent(self.h1))
        self.y0, self.h1 = self.lstm(x0,self.h1)
        self.y1 = self.decoder(self.h1[0])
        
        return self.y1
# can I modify this to set an arbitrary number of layers?

# accuracy and one-hot encoding functions
def onehotTensor(category,n_categories):
    tensor = torch.zeros(1, n_categories,dtype=torch.long)
    tensor[0][category] = 1
    return tensor
        
def get_accuracy(logit, target, batch_size):
  # target.size() and logit have incompatible dimensions?
  print(torch.max(logit, 1)[1])
  print(target.size())
#  print(torch.max(logit, 1)[1].view(target.size()).data) # the formulation of torch.max(logit, 1)[1].view(target.size()).data has an error 
  corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum() # input size is the same as batch size. 
  accuracy = 100.0 * corrects/batch_size
  return accuracy.item()
 

In [131]:
# run Vanilla RNN
# parameters 
N_STEPS = 10
N_INPUTS = 1
N_HIDDEN = 20
N_OUTPUTS = 10
N_EPHOCS = 50

model = PRNN(N_INPUTS,N_HIDDEN,N_OUTPUTS,0.1)
# model = MyModel()
# model.cuda()


optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.9)
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss() # this does not work

train_running_loss = np.zeros(N_EPHOCS)
train_acc = np.zeros(N_EPHOCS)
nparams = 0
for param in model.parameters(): 
  if param.requires_grad:
    nparams += param.data.numpy().size
Phist = np.zeros((nparams,N_EPHOCS))

for epoch in range(N_EPHOCS):
    nps = 0
    running_loss=0
    running_acc=0
    for p,param in enumerate(model.parameters()):
        if param.requires_grad:
            plist = param.data.numpy().flatten()
            for j in range(plist.size):
                Phist[nps,epoch]=plist[j]
                nps+=1
  
    for i, data in enumerate(trainloader):
        
        optimizer.zero_grad()
        x, y_tar = data
#         print(x.shape)
#         print(y_tar.shape)
# #         y_tar = torch.tensor(y_tar, dtype=torch.long)

#         print(y_tar)
#         print(y_tar.size())
        
        y_pred = model(x.view(BATCH_SIZE,N_STEPS,N_INPUTS))
#         print(y_pred.view(BATCH_SIZE,N_OUTPUTS).shape)
        loss = criterion(y_pred.view(BATCH_SIZE,N_OUTPUTS).float(),y_tar.float())
#         print(loss)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        running_acc+=get_accuracy(y_pred.view(BATCH_SIZE,N_OUTPUTS).float(), y_tar.long(), BATCH_SIZE)
    train_running_loss[epoch] = running_loss
    train_acc[epoch] = running_acc/i
    
    print('Epoch:  %d | Loss: %.4f | Train Accuracy: %.2f' %(epoch, train_running_loss[epoch], train_acc[epoch]))

Epoch:  0 | Loss: 3991.0631 | Train Accuracy: 5.23
Epoch:  1 | Loss: 2864.2491 | Train Accuracy: 3.87
Epoch:  2 | Loss: 3457.1776 | Train Accuracy: 5.45
Epoch:  3 | Loss: 2697.7201 | Train Accuracy: 4.42
Epoch:  4 | Loss: 2834.0418 | Train Accuracy: 3.35
Epoch:  5 | Loss: 3142.8443 | Train Accuracy: 4.65
Epoch:  6 | Loss: 2900.7627 | Train Accuracy: 6.45
Epoch:  7 | Loss: 3319.8489 | Train Accuracy: 5.71
Epoch:  8 | Loss: 3118.2663 | Train Accuracy: 4.68
Epoch:  9 | Loss: 2824.9259 | Train Accuracy: 4.42
Epoch:  10 | Loss: 2675.3225 | Train Accuracy: 6.32
Epoch:  11 | Loss: 2787.0011 | Train Accuracy: 5.19
Epoch:  12 | Loss: 3141.9943 | Train Accuracy: 5.45
Epoch:  13 | Loss: 3166.6565 | Train Accuracy: 3.61
Epoch:  14 | Loss: 2918.6796 | Train Accuracy: 5.48
Epoch:  15 | Loss: 2791.3631 | Train Accuracy: 6.26
Epoch:  16 | Loss: 2771.0171 | Train Accuracy: 2.84
Epoch:  17 | Loss: 3029.7013 | Train Accuracy: 4.94
Epoch:  18 | Loss: 2816.1548 | Train Accuracy: 2.58
Epoch:  19 | Loss: 265

In [132]:
# with RNN-LSTM
# run Vanilla RNN
# parameters 
N_STEPS = 10
N_INPUTS = 1
N_HIDDEN = 20
N_OUTPUTS = 10
N_EPHOCS = 50

model = PRNN_LSTM(N_INPUTS,N_HIDDEN,N_OUTPUTS,0.1)
# model = MyModel()
# model.cuda()


optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.9)
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss() # this does not work

train_running_loss = np.zeros(N_EPHOCS)
train_acc = np.zeros(N_EPHOCS)
nparams = 0
for param in model.parameters(): 
  if param.requires_grad:
    nparams += param.data.numpy().size
Phist = np.zeros((nparams,N_EPHOCS))

for epoch in range(N_EPHOCS):
    nps = 0
    running_loss=0
    running_acc=0
    for p,param in enumerate(model.parameters()):
        if param.requires_grad:
            plist = param.data.numpy().flatten()
            for j in range(plist.size):
                Phist[nps,epoch]=plist[j]
                nps+=1
  
    for i, data in enumerate(trainloader):
        
        optimizer.zero_grad()
        x, y_tar = data
#         y_tar = torch.tensor(y_tar, dtype=torch.long)

#         print(y_tar)
#         print(y_tar.size())
        
        y_pred = model(x.view(BATCH_SIZE,N_STEPS,N_INPUTS))
#         print(y_pred.view(BATCH_SIZE,N_OUTPUTS).shape)
        loss = criterion(y_pred.view(BATCH_SIZE,N_OUTPUTS).float(),y_tar.float())
#         print(loss)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        running_acc+=get_accuracy(y_pred.view(BATCH_SIZE,N_OUTPUTS).float(), y_tar.long(), BATCH_SIZE)
    train_running_loss[epoch] = running_loss
    train_acc[epoch] = running_acc/i
    
    print('Epoch:  %d | Loss: %.4f | Train Accuracy: %.2f' %(epoch, train_running_loss[epoch], train_acc[epoch]))

Epoch:  0 | Loss: 3699.7419 | Train Accuracy: 4.97
Epoch:  1 | Loss: 2621.6779 | Train Accuracy: 2.06
Epoch:  2 | Loss: 2694.0456 | Train Accuracy: 2.84
Epoch:  3 | Loss: 2624.5346 | Train Accuracy: 5.74
Epoch:  4 | Loss: 2525.7677 | Train Accuracy: 6.48
Epoch:  5 | Loss: 2675.7388 | Train Accuracy: 5.16
Epoch:  6 | Loss: 2663.7953 | Train Accuracy: 3.10
Epoch:  7 | Loss: 2710.3007 | Train Accuracy: 3.87
Epoch:  8 | Loss: 2596.5698 | Train Accuracy: 3.61
Epoch:  9 | Loss: 2712.3929 | Train Accuracy: 3.61
Epoch:  10 | Loss: 2602.5027 | Train Accuracy: 2.06
Epoch:  11 | Loss: 2699.2000 | Train Accuracy: 2.06
Epoch:  12 | Loss: 2645.8136 | Train Accuracy: 2.32
Epoch:  13 | Loss: 3335.0647 | Train Accuracy: 4.39
Epoch:  14 | Loss: 2860.9734 | Train Accuracy: 4.90
Epoch:  15 | Loss: 2634.0460 | Train Accuracy: 3.10
Epoch:  16 | Loss: 2649.8090 | Train Accuracy: 6.77
Epoch:  17 | Loss: 2966.6416 | Train Accuracy: 5.19
Epoch:  18 | Loss: 2583.3132 | Train Accuracy: 2.58
Epoch:  19 | Loss: 259

Both aren't working very well. Let's try the following:


1.   Use one-hot encoding and go back to CrossEntropy Loss Criterion
2.   Expand the number of input/output characters. 



In [0]:
# switch to one-hot encodings
poem_string_onehot_input = make_onehot_string_tensor(poem_string,addspace=True,add_punctuation=False)

# train/test loader
# generate trainloader and testloader sets
BATCH_SIZE = 96

trainloader = torch.utils.data.DataLoader(poem_string_onehot_input, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

# # download and load testing dataset
# testset = torchvision.datasets.MNIST(root='./data', train=False,
#                                        download=True, transform=transform)
testloader = torch.utils.data.DataLoader(poem_string_onehot_input, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)


In [193]:
# run Vanilla RNN
# parameters 
N_STEPS = 27
N_INPUTS = 10
N_HIDDEN = 10
N_OUTPUTS = 27
N_EPHOCS = 50

# N_STEPS = 28
# N_INPUTS = 28
# N_HIDDEN = 100
# N_OUTPUTS = 10
# N_EPHOCS = 10


model = PRNN(N_INPUTS,N_HIDDEN,N_OUTPUTS,0.1)
# model = MyModel()
# model.cuda()


optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.9)
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss() # this does not work

train_running_loss = np.zeros(N_EPHOCS)
train_acc = np.zeros(N_EPHOCS)
nparams = 0
for param in model.parameters(): 
  if param.requires_grad:
    nparams += param.data.numpy().size
Phist = np.zeros((nparams,N_EPHOCS))

for epoch in range(N_EPHOCS):
    nps = 0
    running_loss=0
    running_acc=0
    for p,param in enumerate(model.parameters()):
        if param.requires_grad:
            plist = param.data.numpy().flatten()
            for j in range(plist.size):
                Phist[nps,epoch]=plist[j]
                nps+=1
  
    for i, data in enumerate(trainloader):
        
        optimizer.zero_grad()
#         print(y_tar)
        x, y_tar = data
                
        y_pred = model(x.view(BATCH_SIZE,N_STEPS,N_INPUTS))
        
#        print(y_tar.shape)
#        print(y_pred.shape) # prediction

        loss = criterion(y_pred.view(BATCH_SIZE,N_OUTPUTS),Variable(y_tar))
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        print(y_pred.view(BATCH_SIZE,N_OUTPUTS))
        # torch.Tensor.view() returns a new tensor with the same data as the self tensor but of a different shape.
        # The returned tensor shares the same data and must have the same number of elements, but may have a different size.        
        # need to fix the .view function!
        running_acc+=get_accuracy(y_pred.view(BATCH_SIZE,N_OUTPUTS).float(), y_tar.long(), BATCH_SIZE)
        
        
    train_running_loss[epoch] = running_loss
    train_acc[epoch] = running_acc/i
    
    print('Epoch:  %d | Loss: %.4f | Train Accuracy: %.2f' %(epoch, train_running_loss[epoch], train_acc[epoch]))

tensor([[ 0.1964, -0.0417, -0.2840,  ..., -0.2828, -0.2611, -0.0805],
        [ 0.2436,  0.0244, -0.3702,  ..., -0.2195, -0.3622, -0.0878],
        [ 0.1410, -0.1064, -0.3047,  ..., -0.3420, -0.2597, -0.0349],
        ...,
        [ 0.2193, -0.1628, -0.2732,  ..., -0.3593, -0.3492, -0.1824],
        [ 0.2271, -0.0628, -0.2515,  ..., -0.2513, -0.2797, -0.1809],
        [ 0.2051, -0.0773, -0.2816,  ..., -0.2578, -0.2570, -0.1178]],
       grad_fn=<ViewBackward>)
tensor([5, 5, 7, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 7, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 7, 5, 5, 5, 5, 5, 5, 7, 7, 5, 5, 5, 5,
        5, 5, 5, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5])
torch.Size([96, 1, 27])


RuntimeError: ignored